In [9]:
import os
import re
import librosa.display 
import glob 
import matplotlib.pyplot as plt
import numpy as np
from os.path import basename

EMOTION = {'anger': 0, 'happiness' : 1, 'sadness' : 2, 'neutral' : 3, 'frustration' : 4, 'excited': 5,
           'fear' : 6,'surprise' : 7,'disgust' : 8, 'other' : 9}


class Input:
    ##spectral, prosody, erergy are dict type
    def __init__(self, spectral, prosody, energy):
        self.spectral = spectral
        self.prosody = prosody
        self.energy = energy
        
    def print(self):
        print("spectral  features: ", spectral)
        print("prosody features: ", prosody)
        print("energy: ", energy)
        
    def input2Vec(self):
        features = []
        s = list(self.spectral.values())
        p = list(self.prosody.values())
        e = list(self.energy.values())
        [features.extend(x) for x in [s, p, e]]
        return features

class Output:
    def __init__(self, duration, code, category_origin, category_evaluation, attribute):
        self.duration = duration
        self.code = code
        self.category_origin = category_origin
        self.category_evaluation = category_evaluation
        self.attribute = attribute
        
    def print(self):
        print("duration: ", self.duration)
        print("code: ", self.code)
        print("category_origin: ", self.category_origin)
        print("category_evaluation: ", self.category_evaluation)
        print("attribute: ", self.attribute)
        
    def output2Vec(self):
        category_evaluation_int = list(map(lambda x: EMOTION[x.lower()], self.category_evaluation))
        return category_evaluation_int

        
def get_input_label_dirs():
    DATA_DIR = "IEMOCAP_full_release"
    NUM_SESSION = 5
    input_output = []
    for i in range (1, NUM_SESSION + 1):
        name_session = "Session" + str(i)
        root_dir_of_wav = DATA_DIR + "/" + name_session + "/sentences" + "/wav"
        root_dir_of_labels = DATA_DIR + "/" + name_session + "/dialog" + "/EmoEvaluation"

        for x in os.walk(root_dir_of_wav):
            if(x[0] == root_dir_of_wav):
                dirs_of_wav = x[1]
                index = -1
            else:
                index = index + 1
                input_output.append((x[0], root_dir_of_labels + "/" + dirs_of_wav[index] + ".txt"))
                
    
    return input_output


def parseOutput(file):
    dict_namefile_output = {}
    # Open file to get all contents excepts the first line.
    f = open(file, 'r')
    content = ""
    index = 0
    for line in f:
        index = index + 1
        if index == 1:
            continue
        content  = content + line
    
    # Find all matched patterns in the content
    ps = re.findall(r'\[.*?\)\n\n', content, re.DOTALL)
    
    # Parse each matched pattern into  'Output' object
    try:
        for p in ps:
            ls = p.split("\n")
            ls = list(filter(lambda x: len(x) > 0 ,ls))
            
            # Split elements of the first line which looks like : 
            # [147.0300 - 151.7101]	Ses01F_impro02_M012	neu	[2.5000, 2.0000, 2.0000]
            ele_line0 = re.search(r'(\[.*?\])(\s)(.*?)(\s)(.*?)(\s)(\[.*?\])', ls[0]).groups()
            
            # Split time components which looks like:
            # [147.0300 - 151.7101]
            time_dur = ele_line0[0]
            ele_time_dur = re.findall(r"[-+]?\d*\.\d+|\d+", time_dur)
            ele_time_dur = [float(x) for x in ele_time_dur]
            
            # Get code and category_origin which looks like:
            # Code: Ses01F_impro02_M012
            # Category_origin: neu
            code = ele_line0[2]
            category_origin = ele_line0[4]
            
            # Split attribute components which looks like:
            # [2.5000, 2.0000, 2.0000]
            attribute = ele_line0[6]
            ele_attribute = re.findall(r"[-+]?\d*\.\d+|\d+", attribute)
            ele_attribute = [float(x) for x in ele_attribute]
            
            # Get categorial_evaluation:
            lines_categorical = list(filter(lambda x : x[0] == 'C', ls))
            rex = re.compile(r'C.*?:(\s)(.*?)(\s)\(.*?\)')
            
            category_evaluation = []
            for l in lines_categorical:
                elements = rex.search(l).groups()
                cat = elements[1]
                cat = cat.split(";")
                cat = map(lambda x: x.lstrip(), cat)
                cat = list(filter(lambda x: len(x)>0, cat))
                category_evaluation.extend(cat)
        
        
            # Make list distinct
            category_evaluation = list(set(category_evaluation))
            
            # Make dict {name_file : parsed_output}
            dict_namefile_output[code] = Output(ele_time_dur, code, category_origin, category_evaluation, ele_attribute)
        return dict_namefile_output
    except Exception as e:
        print(e)
    

def audio2Features(file):
    print(file)
    spectral = {}
    prosody = {}
    energy = {}
    try:
        y, sr = librosa.load(file)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        print(len(spec_bw[0]))
        spectral['min_bandwidth'] = np.min(spec_bw)
        spectral['max_bandwidth'] = np.max(spec_bw)
        ## Todo ..............

        return Input(spectral, prosody, energy)
    except Exception as e:
        print(e)
        print("This is path file cause this error: ", file)
        
    
    
def parseInput(dir):
    dicts = {} 
    # Get features each file in 'directory' and 
    for f in os.listdir(dir):
        if not f.startswith(".") and os.path.splitext(f)[1] == ".wav":
            dicts[os.path.splitext(f)[0]] = audio2Features(dir + "/" + f)

            
    return dicts
    

    
def createInput_Output():
    ds = get_input_label_dirs()
    print("There is ", len(ds), " directories!!")
    input = []
    out = []
    i = 0
    for d in ds:
        i = i + 1
        print("Number dir has been processing: ", i)
        dicts_in = parseInput(d[0])
        dicts_out = parseOutput(d[1])
        
        keys = list(dicts_in.keys())
        for key in keys:
            input.append(dicts_in[key].input2Vec())
            out.append(dicts_out[key].output2Vec())
       
        
    
    print(len(input))
    print(len(out))
    return (input, out)
        
        
    







    
    
#input, output = createInput_Output()
    
    

In [10]:
f1 = r'IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro01/Ses01F_impro01_F006.wav'
f2 = r'IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro01/Ses01F_impro01_F007.wav'
f3 = r'IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro01/Ses01F_impro01_F008.wav'

audio2Features(f1)
audio2Features(f2)




IEMOCAP_full_release\Session1\sentences\wav\Ses01F_impro01\Ses01F_impro01_F006.wav
200
IEMOCAP_full_release\Session1\sentences\wav\Ses01F_impro01\Ses01F_impro01_F007.wav
242
